**Домашнее задание к лекции «Продвинутый pandas»**

Преподаватель: Константин Башевой

**Задание 1**

Для датафрейма log из материалов занятия создайте столбец source_type по следующим правилам:
*   если источник traffic_source равен yandex или google, то в source_type ставится organic \
*   для источников paid и email из России - ставим ad \
*   для источников paid и email не из России - ставим other \
*   все остальные варианты берем из traffic_source без изменений





In [58]:
import pandas as pd

In [59]:
df_log = pd.read_csv('visit_log.csv', sep = ';')
df_log.head()

,timestamp,visit_id,url,region,user_id,traffic_source
0,1549980692,e3b0c44298,https://host.ru/3c19b4ef7371864fa3,Russia,b1613cc09f,yandex
1,1549980704,6e340b9cff,https://host.ru/c8d9213a31839f9a3a,Russia,4c3ec14bee,direct
2,1549980715,96a296d224,https://host.ru/b8b58337d272ee7b15,Russia,a8c40697fb,yandex
3,1549980725,709e80c884,https://host.ru/b8b58337d272ee7b15,Russia,521ac1d6a0,yandex
4,1549980736,df3f619804,https://host.ru/b8b58337d272ee7b15,Russia,d7323c571c,yandex


In [60]:
log = df_log.loc[:, ['region', 'traffic_source']]
log.head()

,region,traffic_source
0,Russia,yandex
1,Russia,direct
2,Russia,yandex
3,Russia,yandex
4,Russia,yandex


In [75]:
# если источник traffic_source равен yandex или google, то в source_type ставится organic
log.loc[(log.traffic_source == 'yandex')|
        (log.traffic_source == 'google'), 'source_type'] = 'organic'

# для источников paid и email из России - ставим ad
log.loc[((log.traffic_source == 'email')|
         (log.traffic_source == 'paid')) & 
        (log.region == 'Russia'), 'source_type'] = 'ad'

# для источников paid и email не из России - ставим other
log.loc[((log.traffic_source == 'paid')|
         (log.traffic_source == 'email')) & 
        ~(log.region == 'Russia'), 'source_type'] = 'other'

# все остальные варианты берем из traffic_source без изменений
log.loc[pd.isnull(log.source_type), 'source_type'] = log.traffic_source # всем пустым значениям присваиваю значения из traffic_source

log.head(200)

,region,traffic_source,source_type
0,Russia,yandex,organic
1,Russia,direct,direct
2,Russia,yandex,organic
3,Russia,yandex,organic
4,Russia,yandex,organic
...,...,...,...
195,Russia,email,ad
196,Russia,paid,ad
197,Belarus,paid,other
198,Belarus,paid,other


**Задание 2**

В файле URLs.txt содержатся url страниц новостного сайта. Вам необходимо отфильтровать его по адресам страниц с текстами новостей. Известно, что шаблон страницы новостей имеет внутри url следующую конструкцию: /, затем 8 цифр, затем дефис. Выполните следующие действия:

1.   Прочитайте содержимое файла с датафрейм
2.   Отфильтруйте страницы с текстом новостей, используя метод str.contains и регулярное выражение в соответствии с заданным шаблоном





In [82]:
import re

In [83]:
df_url = pd.read_table('URLs.txt',sep='\s+', engine='python')
df_url.head()

,url
0,/world/
1,/latest/
2,/?updated=top
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...


In [84]:
df_url[df_url.url.str.contains(r'/\d{8}-')].head()

,url
3,/politics/36188461-s-marta-zhizn-rossiyan-susc...
4,/world/36007585-tramp-pridumal-kak-reshit-ukra...
5,/science/36157853-nasa-sobiraet-ekstrennuyu-pr...
6,/video/36001498-poyavilis-pervye-podrobnosti-g...
7,/world/36007585-tramp-pridumal-kak-reshit-ukra...


**Задание 3**

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.

In [85]:
df_ratings = pd.read_csv('ratings.csv')
df_ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [86]:
df_ratings['date'] = pd.to_datetime(df_ratings['timestamp'], unit = 's')
df_ratings.head()

,userId,movieId,rating,timestamp,date
0,1,31,2.5,1260759144,2009-12-14 02:52:24
1,1,1029,3.0,1260759179,2009-12-14 02:52:59
2,1,1061,3.0,1260759182,2009-12-14 02:53:02
3,1,1129,2.0,1260759185,2009-12-14 02:53:05
4,1,1172,4.0,1260759205,2009-12-14 02:53:25


Создаю функцию по подсчёту времени жизни пользователя

In [87]:
def lifetime(param):
        return param.max() - param.min()

Считаю количество оценок пользователей

In [96]:
df_users = df_ratings.groupby('userId')['rating'].count().reset_index()
df_users.head()

,userId,rating
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100


Создаю таблицу с активными пользователями, у которых кол-во оценок больше 100

In [98]:
df_active_users = df_users.loc[df_users.rating > 100]
df_active_users.head()

,userId,rating
3,4,204
7,8,116
14,15,1700
16,17,363
18,19,423


Объединяю таблицы, оставляя только активных пользователей со всеми данными

In [101]:
df_users_ratings = df_active_users.merge(df_ratings, on = 'userId', how = 'left')
df_users_ratings.head()

,userId,rating_x,movieId,rating_y,timestamp,date
0,4,204,10,4.0,949810645,2000-02-06 04:17:25
1,4,204,34,5.0,949919556,2000-02-07 10:32:36
2,4,204,112,5.0,949810582,2000-02-06 04:16:22
3,4,204,141,5.0,949919681,2000-02-07 10:34:41
4,4,204,153,4.0,949811346,2000-02-06 04:29:06


In [103]:
df_lifetime_act_users = df_users_ratings.groupby('userId').agg({'date': lifetime})
df_lifetime_act_users.head()

,date
userId,
4,2 days 08:32:40
8,0 days 23:39:47
15,5455 days 22:38:16
17,0 days 02:14:13
19,0 days 01:28:02


Рассчитываю среднее

In [134]:
lifetime_mean = df_lifetime_act_users.mean()['date']
print(f'Среднее время жизни пользователей, которые выставили более 100 оценок: {lifetime_mean}')

Среднее время жизни пользователей, которые выставили более 100 оценок: 463 days 21:28:27.449612408


**Задание 4**

Дана статистика услуг перевозок клиентов компании по типам (см. файл “Python_13_join.ipynb” в разделе Материалы для лекции «Продвинутый pandas» ---- Ноутбуки к лекции «Продвинутый pandas»).\
Необходимо сформировать две таблицы:

*   таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
*   аналогичную таблицу по типам выручки с указанием адреса клиента

Дана статистика услуг перевозок клиентов компании по типам:

*   rzd - железнодорожные перевозки
*   auto - автомобильные перевозки
*   air - воздушные перевозки
*   client_base - адреса клиентов

In [106]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [107]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [108]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [109]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [110]:
from functools import reduce

Таблица по типам выручки с указанием адреса клиента

In [130]:
revenue_types_base = reduce (lambda a,b: a.merge(b, how = 'outer', on = 'client_id'), [client_base, auto, rzd, air])
revenue_types_base

,client_id,address,auto_revenue,rzd_revenue,air_revenue
0,111,Комсомольская 4,NaN,1093.0,NaN
1,112,Энтузиастов 8а,NaN,2810.0,NaN
2,113,Левобережная 1а,57483.0,10283.0,NaN
3,114,Мира 14,83.0,5774.0,NaN
4,115,ЗЖБИиДК 1,912.0,981.0,81.0
5,116,Строителей 18,4834.0,NaN,4.0
6,117,Панфиловская 33,98.0,NaN,13.0
7,118,Мастеркова 4,NaN,NaN,173.0


Таблица с тремя типами выручки для каждого client_id без указания адреса клиента

In [131]:
revenue_types = reduce (lambda a,b: a.merge(b, how = 'outer', on = 'client_id'), [rzd, auto, air])
revenue_types

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0
